In [4]:
%matplotlib inline
from skimage import data, io, filters
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

## purpose

* upload sketches to S3
* build stimulus dictionary and write to database

## upload sketches to s3

In [1]:
upload_dir = './sketch'

In [2]:
import boto
runThis = 0
if runThis:
    conn = boto.connect_s3()
    b = conn.create_bucket('sketchpad_basic_pilot2_sketches')
    all_files = [i for i in os.listdir(upload_dir) if i != '.DS_Store']
    for a in all_files:
        print a
        k = b.new_key(a)
        k.set_contents_from_filename(os.path.join(upload_dir,a))
        k.set_acl('public-read')

## build stimulus dictionary

In [8]:
## read in experimental metadata file
path_to_metadata = '../../analysis/sketchpad_basic_pilot2_group_data.csv'
meta = pd.read_csv(path_to_metadata)

In [9]:
## clean up and add filename column
meta2 = meta.drop(['svg','png','Unnamed: 0'],axis=1)
filename = []
games = []
for i,row in meta2.iterrows():
    filename.append('gameID_{}_trial_{}.png'.format(row['gameID'],row['trialNum']))
    games.append([])
meta2['filename'] = filename
meta2['games'] = games

In [22]:
## write out metadata to json file
stimdict = meta2.to_dict(orient='records')
import json
with open('sketchpad_basic_recog_meta.js', 'w') as fout:
    json.dump(stimdict, fout)

In [12]:
J = json.loads(open('sketchpad_basic_recog_meta.js',mode='ru').read())
assert len(J)==len(meta2)

In [13]:
'{} unique games.'.format(len(np.unique(meta2.gameID.values)))

'93 unique games.'

### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [26]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [27]:
db = conn['stimuli']
coll = db['sketchpad_basic_pilot2_sketches']

In [28]:
## actually add data now to the database
for (i,j) in enumerate(J):
    if i%100==0:
        print ('%d of %d' % (i,len(J)))
    coll.insert_one(j)

0 of 2976
100 of 2976
200 of 2976
300 of 2976
400 of 2976
500 of 2976
600 of 2976
700 of 2976
800 of 2976
900 of 2976
1000 of 2976
1100 of 2976
1200 of 2976
1300 of 2976
1400 of 2976
1500 of 2976
1600 of 2976
1700 of 2976
1800 of 2976
1900 of 2976
2000 of 2976
2100 of 2976
2200 of 2976
2300 of 2976
2400 of 2976
2500 of 2976
2600 of 2976
2700 of 2976
2800 of 2976
2900 of 2976


In [29]:
coll.find_one()

{u'Distractor1': u'crow',
 u'Distractor2': u'redsport',
 u'Distractor3': u'basset',
 u'_id': ObjectId('5a84f478a5319bcb0bb66227'),
 u'category': u'chair',
 u'condition': u'further',
 u'drawDuration': 18.389,
 u'filename': u'gameID_7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5_trial_1.png',
 u'gameID': u'7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5',
 u'games': [],
 u'iteration': u'pilot2',
 u'mean_intensity': 0.0347,
 u'numStrokes': 7,
 u'outcome': True,
 u'pose': 35,
 u'response': u'knob',
 u'svgStringLength': 566,
 u'svgStringLengthPerStroke': 80.85714285714286,
 u'svgStringStd': 22.87499302999782,
 u'target': u'knob',
 u'trialNum': 1,
 u'viewerRT': 6.656000000000001}

In [41]:
a = list(coll.aggregate([
        { '$addFields' : { 'numGames': { "$size": '$games'} } }, 
        { '$sort' : { 'numGames' : 1} },  
        { '$limit' : 10 }
    ]))
        

In [43]:
a

[{u'Distractor1': u'leather',
  u'Distractor2': u'inlay',
  u'Distractor3': u'squat',
  u'_id': ObjectId('5a84f478a5319bcb0bb6622f'),
  u'category': u'chair',
  u'condition': u'closer',
  u'drawDuration': 21.246,
  u'filename': u'gameID_7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5_trial_9.png',
  u'gameID': u'7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5',
  u'games': [],
  u'iteration': u'pilot2',
  u'mean_intensity': 0.0722,
  u'numGames': 0,
  u'numStrokes': 11,
  u'outcome': True,
  u'pose': 35,
  u'response': u'waiting',
  u'svgStringLength': 1950,
  u'svgStringLengthPerStroke': 177.27272727272728,
  u'svgStringStd': 214.87590114426976,
  u'target': u'waiting',
  u'trialNum': 9,
  u'viewerRT': 1.7930000000000001},
 {u'Distractor1': u'leather',
  u'Distractor2': u'waiting',
  u'Distractor3': u'squat',
  u'_id': ObjectId('5a84f478a5319bcb0bb66230'),
  u'category': u'chair',
  u'condition': u'closer',
  u'drawDuration': 25.374000000000002,
  u'filename': u'gameID_7004-6ff0964c-ff95-40f9-869